In [1]:
import numpy as np
import pandas as pd

# Extract parameters from your config file
class PINNACLEParams:
    def __init__(self):
        # Physical constants
        self.F = 96485  # Faraday constant [C/mol]
        self.R = 8.3145  # Gas constant [J/(mol·K)]
        self.T = 293  # Temperature [K]
        
        # Diffusion coefficients [m²/s]
        self.D_cv = 1.0e-21
        self.D_av = 1.0e-21
        self.D_h = 3.2823e-4
        
        # Mobility coefficients [m²/(V·s)]
        self.U_cv = -1.0562e-19  # Note: taking absolute value for calculations
        self.U_av = 7.9212e-20
        self.U_h = 0.013
        
        # Permittivities [F/m]
        self.eps_film = 1.239e-10
        
        # Characteristic scales (your proposed values)
        self.L_c = 1e-9  # Steady state film thickness [m]
        self.phi_c = self.R * self.T / self.F  # Thermal voltage [V]
        self.c_c_cv = 1e-5  # Characteristic CV concentration [mol/m³]
        self.c_c_av = 1e-5  # Characteristic AV concentration [mol/m³]
        self.c_c_h = 4.1683e-4  # Initial hole concentration [mol/m³]
        
        # Geometric mean diffusion time
        self.t_c = ((self.L_c**2 / self.D_cv) * 
                   (self.L_c**2 / self.D_av) * 
                   (self.L_c**2 / self.D_h))**(1/3)
        
        # Rate constants for Damköhler numbers
        self.k2_0 = 3.6e-6  # [mol/(m²·s)]
        self.k5_0 = 7.65e-9  # [mol/(m²·s)]
        self.Omega = 1.4e-5  # Molar volume [m³/mol]
        
        # Species charges
        self.z_cv = -8/3
        self.z_av = 2

def calculate_dimensionless_groups(params):
    """Calculate all key dimensionless parameters"""
    
    results = {}
    
    # 1. Péclet Numbers (Convection vs Diffusion)
    results['Pe_CV'] = (abs(params.U_cv) * params.phi_c * params.t_c) / params.L_c
    results['Pe_AV'] = (params.U_av * params.phi_c * params.t_c) / params.L_c
    results['Pe_h'] = params.F * params.phi_c * params.L_c / (params.R * params.T)  # = 1 by design
    
    # 2. Charge Density Parameter (Electrostatic vs Diffusion)
    # Using CV concentration as representative
    results['Lambda_CV'] = (params.L_c**2 * params.c_c_cv * params.F) / (params.phi_c * params.eps_film)
    results['Lambda_AV'] = (params.L_c**2 * params.c_c_av * params.F) / (params.phi_c * params.eps_film)
    
    # 3. Film Growth Damköhler Number
    k_diff = params.k2_0 - params.k5_0  # Net growth rate
    results['Da_growth'] = (params.t_c * params.Omega * abs(k_diff)) / params.L_c
    
    # 4. Diffusion Time Scale Ratios
    tau_cv = params.L_c**2 / params.D_cv
    tau_av = params.L_c**2 / params.D_av  
    tau_h = params.L_c**2 / params.D_h
    
    results['tau_CV'] = tau_cv
    results['tau_AV'] = tau_av
    results['tau_h'] = tau_h
    results['tau_ratio_CV_AV'] = tau_cv / tau_av
    results['tau_ratio_h_CV'] = tau_h / tau_cv
    
    # 5. Mobility Ratios
    results['mobility_ratio_CV_AV'] = abs(params.U_cv) / params.U_av
    
    # 6. Check the 10^7 scaling in initial condition
    results['IC_scaling_factor'] = 1e7 * params.L_c / params.phi_c
    
    # 7. Key coefficients from your equations
    results['D_coeff_CV'] = params.D_cv * params.t_c / params.L_c**2
    results['D_coeff_AV'] = params.D_av * params.t_c / params.L_c**2
    results['D_coeff_h'] = params.D_h * params.t_c / params.L_c**2
    
    results['U_coeff_CV'] = abs(params.U_cv) * params.t_c * params.phi_c / params.L_c**2
    results['U_coeff_AV'] = params.U_av * params.t_c * params.phi_c / params.L_c**2
    
    return results

def analyze_physics_regimes(results):
    """Analyze which physics regimes your system is in"""
    
    analysis = {}
    
    # Péclet number analysis
    if results['Pe_CV'] < 0.1:
        analysis['CV_transport'] = "Diffusion-dominated"
    elif results['Pe_CV'] > 10:
        analysis['CV_transport'] = "Convection-dominated" 
    else:
        analysis['CV_transport'] = "Mixed transport"
        
    if results['Pe_AV'] < 0.1:
        analysis['AV_transport'] = "Diffusion-dominated"
    elif results['Pe_AV'] > 10:
        analysis['AV_transport'] = "Convection-dominated"
    else:
        analysis['AV_transport'] = "Mixed transport"
    
    # Electrostatic analysis
    if results['Lambda_CV'] < 0.1:
        analysis['electrostatics'] = "Weak space charge (can use electroneutrality)"
    elif results['Lambda_CV'] > 10:
        analysis['electrostatics'] = "Strong space charge (must solve Poisson)"
    else:
        analysis['electrostatics'] = "Moderate space charge (coupled problem)"
    
    # Film growth analysis  
    if results['Da_growth'] < 0.1:
        analysis['film_growth'] = "Quasi-steady (slow growth vs diffusion)"
    elif results['Da_growth'] > 10:
        analysis['film_growth'] = "Moving boundary dominated"
    else:
        analysis['film_growth'] = "Coupled growth-transport"
    
    # Time scale analysis
    if results['tau_ratio_h_CV'] < 0.01:
        analysis['time_scales'] = "Hole transport much faster (stiff system)"
    elif results['tau_ratio_h_CV'] > 100:
        analysis['time_scales'] = "Vacancy transport much faster"
    else:
        analysis['time_scales'] = "Comparable time scales"
    
    return analysis

def create_summary_table(results, analysis):
    """Create a formatted summary table"""
    
    print("="*80)
    print("                    nexPINNACLE DIMENSIONLESS ANALYSIS")
    print("="*80)
    
    print(f"\nCHARACTERISTIC SCALES:")
    print(f"  Length scale (L_c):           {params.L_c:.1e} m")
    print(f"  Time scale (t_c):             {params.t_c:.1e} s") 
    print(f"  Potential scale (φ_c):        {params.phi_c:.4f} V")
    print(f"  Concentration scale (CV/AV):  {params.c_c_cv:.1e} mol/m³")
    print(f"  Concentration scale (h):      {params.c_c_h:.1e} mol/m³")
    
    print(f"\nKEY DIMENSIONLESS GROUPS:")
    print(f"  Péclet CV (Pe_CV):            {results['Pe_CV']:.2e}")
    print(f"  Péclet AV (Pe_AV):            {results['Pe_AV']:.2e}")
    print(f"  Péclet h (Pe_h):              {results['Pe_h']:.2f}")
    print(f"  Charge density (Λ_CV):        {results['Lambda_CV']:.2e}")
    print(f"  Charge density (Λ_AV):        {results['Lambda_AV']:.2e}")
    print(f"  Film growth (Da_growth):      {results['Da_growth']:.2e}")
    
    print(f"\nTIME SCALE COMPARISON:")
    print(f"  CV diffusion time:            {results['tau_CV']:.2e} s")
    print(f"  AV diffusion time:            {results['tau_AV']:.2e} s")
    print(f"  Hole diffusion time:          {results['tau_h']:.2e} s")
    print(f"  Ratio τ_h/τ_CV:               {results['tau_ratio_h_CV']:.2e}")
    
    print(f"\nEQUATION COEFFICIENTS (should be O(1)):")
    print(f"  Diffusion coeff CV:           {results['D_coeff_CV']:.2f}")
    print(f"  Diffusion coeff AV:           {results['D_coeff_AV']:.2f}")
    print(f"  Diffusion coeff h:            {results['D_coeff_h']:.2e}")
    print(f"  Convection coeff CV:          {results['U_coeff_CV']:.2e}")
    print(f"  Convection coeff AV:          {results['U_coeff_AV']:.2e}")
    
    print(f"\nSCALING CHECKS:")
    print(f"  10^7 scaling factor:          {results['IC_scaling_factor']:.2f} (should be O(1))")
    
    print(f"\nPHYSICS REGIME ANALYSIS:")
    for key, value in analysis.items():
        print(f"  {key.replace('_', ' ').title():<25}: {value}")
    
    print("\n" + "="*80)

# Main execution
if __name__ == "__main__":
    # Initialize parameters
    params = PINNACLEParams()
    
    # Calculate dimensionless groups
    results = calculate_dimensionless_groups(params)
    
    # Analyze physics regimes
    analysis = analyze_physics_regimes(results)
    
    # Create summary
    create_summary_table(results, analysis)
    
    # Additional insights for PINN implementation
    print("\nPINN IMPLEMENTATION INSIGHTS:")
    print("-" * 40)
    
    if results['tau_ratio_h_CV'] < 0.01:
        print("⚠️  STIFF SYSTEM: Hole transport >> vacancy transport")
        print("   → Consider separate time scales or implicit schemes")
    
    if results['Pe_CV'] > 1 or results['Pe_AV'] > 1:
        print("⚠️  CONVECTION-DOMINATED: Sharp gradients possible")
        print("   → May need attention to network architecture")
        
    if results['Lambda_CV'] > 1:
        print("✓  STRONG ELECTROSTATICS: Poisson equation is crucial")
        print("   → Don't neglect space charge effects")
    
    if results['Da_growth'] > 0.1:
        print("✓  MOVING BOUNDARY IMPORTANT: Film growth matters")
        print("   → Full moving boundary treatment needed")
        
    if results['IC_scaling_factor'] > 2 or results['IC_scaling_factor'] < 0.5:
        print("⚠️  INITIAL CONDITION SCALING: Check φ_c choice")
    else:
        print("✓  INITIAL CONDITION SCALING: Well-scaled!")

                    nexPINNACLE DIMENSIONLESS ANALYSIS

CHARACTERISTIC SCALES:
  Length scale (L_c):           1.0e-09 m
  Time scale (t_c):             1.4e-03 s
  Potential scale (φ_c):        0.0252 V
  Concentration scale (CV/AV):  1.0e-05 mol/m³
  Concentration scale (h):      4.2e-04 mol/m³

KEY DIMENSIONLESS GROUPS:
  Péclet CV (Pe_CV):            3.87e-15
  Péclet AV (Pe_AV):            2.90e-15
  Péclet h (Pe_h):              0.00
  Charge density (Λ_CV):        3.08e-07
  Charge density (Λ_AV):        3.08e-07
  Film growth (Da_growth):      7.29e-05

TIME SCALE COMPARISON:
  CV diffusion time:            1.00e+03 s
  AV diffusion time:            1.00e+03 s
  Hole diffusion time:          3.05e-15 s
  Ratio τ_h/τ_CV:               3.05e-18

EQUATION COEFFICIENTS (should be O(1)):
  Diffusion coeff CV:           0.00
  Diffusion coeff AV:           0.00
  Diffusion coeff h:            4.76e+11
  Convection coeff CV:          3.87e-06
  Convection coeff AV:          2.90e-06



In [2]:
import numpy as np
import pandas as pd

# Extract parameters from your config file
class PINNACLEParams:
    def __init__(self):
        # Physical constants
        self.F = 96485  # Faraday constant [C/mol]
        self.R = 8.3145  # Gas constant [J/(mol·K)]
        self.T = 293  # Temperature [K]
        
        # Diffusion coefficients [m²/s]
        self.D_cv = 1.0e-21
        self.D_av = 1.0e-21
        self.D_h = 3.2823e-4
        
        # Mobility coefficients [m²/(V·s)]
        self.U_cv = -1.0562e-19  # Note: taking absolute value for calculations
        self.U_av = 7.9212e-20
        self.U_h = 0.013
        
        # Permittivities [F/m]
        self.eps_film = 1.239e-10
        
        # Characteristic scales (your proposed values)
        self.L_c = 1e-9  # Steady state film thickness [m]
        self.phi_c = self.R * self.T / self.F  # Thermal voltage [V]
        self.c_c_cv = 1e-5  # Characteristic CV concentration [mol/m³]
        self.c_c_av = 1e-5  # Characteristic AV concentration [mol/m³]
        self.c_c_h = 4.1683e-4  # Initial hole concentration [mol/m³]
        
        # Separate time scales for different physics
        self.t_c_vacancy = self.L_c**2 / self.D_cv  # CV diffusion time (also ≈ AV time since D_cv ≈ D_av)
        self.t_c_hole = self.L_c**2 / self.D_h      # Hole diffusion time
        
        # Primary time scale for vacancy transport (main physics)
        self.t_c = self.t_c_vacancy
        
        # Rate constants for Damköhler numbers
        self.k2_0 = 3.6e-6  # [mol/(m²·s)]
        self.k5_0 = 7.65e-9  # [mol/(m²·s)]
        self.Omega = 1.4e-5  # Molar volume [m³/mol]
        
        # Species charges
        self.z_cv = -8/3
        self.z_av = 2

def calculate_dimensionless_groups(params):
    """Calculate all key dimensionless parameters"""
    
    results = {}
    
    # 1. Péclet Numbers (Convection vs Diffusion)
    results['Pe_CV'] = abs(params.U_cv) * params.phi_c * params.L_c / params.D_cv
    results['Pe_AV'] = params.U_av * params.phi_c * params.L_c / params.D_av
    results['Pe_h'] = params.F * params.phi_c * params.L_c / (params.R * params.T)  # = 1 by design
    
    # 2. Charge Density Parameter (Electrostatic vs Diffusion)
    # Using CV concentration as representative
    results['Lambda_CV'] = (params.L_c**2 * params.c_c_cv * params.F) / (params.phi_c * params.eps_film)
    results['Lambda_AV'] = (params.L_c**2 * params.c_c_av * params.F) / (params.phi_c * params.eps_film)
    
    # 3. Film Growth Damköhler Number
    k_diff = params.k2_0 - params.k5_0  # Net growth rate
    results['Da_growth'] = (params.t_c * params.Omega * abs(k_diff)) / params.L_c
    
    # 4. Time Scale Analysis (separate scales)
    tau_cv = params.L_c**2 / params.D_cv
    tau_av = params.L_c**2 / params.D_av  
    tau_h = params.L_c**2 / params.D_h
    
    results['tau_CV'] = tau_cv
    results['tau_AV'] = tau_av
    results['tau_h'] = tau_h
    results['tau_ratio_CV_AV'] = tau_cv / tau_av
    results['tau_ratio_h_vacancy'] = tau_h / tau_cv
    results['time_scale_separation'] = params.t_c_vacancy / params.t_c_hole
    
    # 5. Mobility Ratios
    results['mobility_ratio_CV_AV'] = abs(params.U_cv) / params.U_av
    
    # 6. Check the 10^7 scaling in initial condition
    results['IC_scaling_factor'] = 1e7 * params.L_c / params.phi_c
    
    # 7. Key coefficients using VACANCY time scale
    results['D_coeff_CV'] = params.D_cv * params.t_c / params.L_c**2
    results['D_coeff_AV'] = params.D_av * params.t_c / params.L_c**2
    
    # Hole coefficients using HOLE time scale  
    results['D_coeff_h_on_hole_scale'] = params.D_h * params.t_c_hole / params.L_c**2
    results['D_coeff_h_on_vacancy_scale'] = params.D_h * params.t_c / params.L_c**2
    
    results['U_coeff_CV'] = abs(params.U_cv) * params.t_c * params.phi_c / params.L_c**2
    results['U_coeff_AV'] = params.U_av * params.t_c * params.phi_c / params.L_c**2
    
    return results

def analyze_physics_regimes(results):
    """Analyze which physics regimes your system is in"""
    
    analysis = {}
    
    # Péclet number analysis
    if results['Pe_CV'] < 0.1:
        analysis['CV_transport'] = "Diffusion-dominated"
    elif results['Pe_CV'] > 10:
        analysis['CV_transport'] = "Convection-dominated" 
    else:
        analysis['CV_transport'] = "Mixed transport"
        
    if results['Pe_AV'] < 0.1:
        analysis['AV_transport'] = "Diffusion-dominated"
    elif results['Pe_AV'] > 10:
        analysis['AV_transport'] = "Convection-dominated"
    else:
        analysis['AV_transport'] = "Mixed transport"
    
    # Electrostatic analysis
    if results['Lambda_CV'] < 0.1:
        analysis['electrostatics'] = "Weak space charge (can use electroneutrality)"
    elif results['Lambda_CV'] > 10:
        analysis['electrostatics'] = "Strong space charge (must solve Poisson)"
    else:
        analysis['electrostatics'] = "Moderate space charge (coupled problem)"
    
    # Film growth analysis  
    if results['Da_growth'] < 0.1:
        analysis['film_growth'] = "Quasi-steady (slow growth vs diffusion)"
    elif results['Da_growth'] > 10:
        analysis['film_growth'] = "Moving boundary dominated"
    else:
        analysis['film_growth'] = "Coupled growth-transport"
    
    # Time scale analysis
    if results['tau_ratio_h_vacancy'] < 0.01:
        analysis['time_scales'] = "Hole transport much faster (stiff system)"
    elif results['tau_ratio_h_vacancy'] > 100:
        analysis['time_scales'] = "Vacancy transport much faster"
    else:
        analysis['time_scales'] = "Comparable time scales"
    
    return analysis

def create_summary_table(results, analysis):
    """Create a formatted summary table"""
    
    print("="*80)
    print("                    nexPINNACLE DIMENSIONLESS ANALYSIS")
    print("="*80)
    
    print(f"\nCHARACTERISTIC SCALES:")
    print(f"  Length scale (L_c):           {params.L_c:.1e} m")
    print(f"  Vacancy time scale (t_c):     {params.t_c_vacancy:.1e} s") 
    print(f"  Hole time scale:              {params.t_c_hole:.1e} s")
    print(f"  Time scale separation:        {params.t_c_vacancy/params.t_c_hole:.1e}")
    print(f"  Potential scale (φ_c):        {params.phi_c:.4f} V")
    print(f"  Concentration scale (CV/AV):  {params.c_c_cv:.1e} mol/m³")
    print(f"  Concentration scale (h):      {params.c_c_h:.1e} mol/m³")
    
    print(f"\nKEY DIMENSIONLESS GROUPS:")
    print(f"  Péclet CV (Pe_CV):            {results['Pe_CV']:.2e}")
    print(f"  Péclet AV (Pe_AV):            {results['Pe_AV']:.2e}")
    print(f"  Péclet h (Pe_h):              {results['Pe_h']:.2f}")
    print(f"  Charge density (Λ_CV):        {results['Lambda_CV']:.2e}")
    print(f"  Charge density (Λ_AV):        {results['Lambda_AV']:.2e}")
    print(f"  Film growth (Da_growth):      {results['Da_growth']:.2e}")
    
    print(f"\nTIME SCALE COMPARISON:")
    print(f"  CV diffusion time:            {results['tau_CV']:.2e} s")
    print(f"  AV diffusion time:            {results['tau_AV']:.2e} s")
    print(f"  Hole diffusion time:          {results['tau_h']:.2e} s")
    print(f"  Ratio τ_h/τ_vacancy:          {results['tau_ratio_h_vacancy']:.2e}")
    print(f"  Time scale separation:        {results['time_scale_separation']:.2e}")
    
    print(f"\nEQUATION COEFFICIENTS ON PROPER SCALES:")
    print(f"  [VACANCY TRANSPORT - should be O(1)]")
    print(f"    Diffusion coeff CV:         {results['D_coeff_CV']:.2f}")
    print(f"    Diffusion coeff AV:         {results['D_coeff_AV']:.2f}")
    print(f"    Convection coeff CV:        {results['U_coeff_CV']:.2e}")
    print(f"    Convection coeff AV:        {results['U_coeff_AV']:.2e}")
    print(f"  [HOLE TRANSPORT]")
    print(f"    On hole time scale:         {results['D_coeff_h_on_hole_scale']:.2f}")
    print(f"    On vacancy time scale:      {results['D_coeff_h_on_vacancy_scale']:.2e}")
    
    print(f"\nSCALING CHECKS:")
    print(f"  10^7 scaling factor:          {results['IC_scaling_factor']:.2f} (should be O(1))")
    
    print(f"\nPHYSICS REGIME ANALYSIS:")
    for key, value in analysis.items():
        print(f"  {key.replace('_', ' ').title():<25}: {value}")
    
    print("\n" + "="*80)

# Main execution
if __name__ == "__main__":
    # Initialize parameters
    params = PINNACLEParams()
    
    # Calculate dimensionless groups
    results = calculate_dimensionless_groups(params)
    
    # Analyze physics regimes
    analysis = analyze_physics_regimes(results)
    
    # Create summary
    create_summary_table(results, analysis)
    
    # Additional insights for PINN implementation
    print("\nPINN IMPLEMENTATION INSIGHTS:")
    print("-" * 40)
    
    if results['tau_ratio_h_vacancy'] < 0.01:
        print("⚠️  STIFF SYSTEM: Hole transport >> vacancy transport")
        print("   → Consider separate time scales or quasi-steady holes")
        print("   → Hole diffusion coeff on vacancy scale: {:.1e}".format(results['D_coeff_h_on_vacancy_scale']))
    
    if results['Pe_CV'] > 1 or results['Pe_AV'] > 1:
        print("⚠️  CONVECTION-DOMINATED: Sharp gradients possible")
        print("   → May need attention to network architecture")
        
    if results['Lambda_CV'] > 1:
        print("✓  STRONG ELECTROSTATICS: Poisson equation is crucial")
        print("   → Don't neglect space charge effects")
    
    if results['Da_growth'] > 0.1:
        print("✓  MOVING BOUNDARY IMPORTANT: Film growth matters")
        print("   → Full moving boundary treatment needed")
        
    if results['IC_scaling_factor'] > 2 or results['IC_scaling_factor'] < 0.5:
        print("⚠️  INITIAL CONDITION SCALING: Check φ_c choice")
    else:
        print("✓  INITIAL CONDITION SCALING: Well-scaled!")

                    nexPINNACLE DIMENSIONLESS ANALYSIS

CHARACTERISTIC SCALES:
  Length scale (L_c):           1.0e-09 m
  Vacancy time scale (t_c):     1.0e+03 s
  Hole time scale:              3.0e-15 s
  Time scale separation:        3.3e+17
  Potential scale (φ_c):        0.0252 V
  Concentration scale (CV/AV):  1.0e-05 mol/m³
  Concentration scale (h):      4.2e-04 mol/m³

KEY DIMENSIONLESS GROUPS:
  Péclet CV (Pe_CV):            2.67e-09
  Péclet AV (Pe_AV):            2.00e-09
  Péclet h (Pe_h):              0.00
  Charge density (Λ_CV):        3.08e-07
  Charge density (Λ_AV):        3.08e-07
  Film growth (Da_growth):      5.03e+01

TIME SCALE COMPARISON:
  CV diffusion time:            1.00e+03 s
  AV diffusion time:            1.00e+03 s
  Hole diffusion time:          3.05e-15 s
  Ratio τ_h/τ_vacancy:          3.05e-18
  Time scale separation:        3.28e+17

EQUATION COEFFICIENTS ON PROPER SCALES:
  [VACANCY TRANSPORT - should be O(1)]
    Diffusion coeff CV:         1.00